In [1]:
import os
import numpy as np
from dotenv import load_dotenv
from openai import AzureOpenAI
from pprint import pprint
from utils.prompts.prompt import *
from utils.prompts.Schema_Prompt_v1 import *
from utils.postgres_functions import *

In [2]:
load_dotenv()

True

In [3]:
def calculate_confidence_score(logprobs: list) -> float:
    probs = []
    # iterate through the token predictions and get the linear probs [0;1]
    for logprob in logprobs:
        probs.append(np.round(np.exp(logprob)*100, 2))
    # calculate the confidence
    return round(np.mean(probs), 2)

In [4]:
client = AzureOpenAI(
    api_key = str(os.environ['azure_api_key']),
    api_version = "2024-03-01-preview",
    azure_endpoint = str(os.environ['azure_api_endpoint'])
)

In [16]:
# QUERY = "Forgalomban van a novocetrin?"
# QUERY = "Milyen ciszplatin hatóanyagú gyógyszert tudok filmtablettás gyógyszerként kiírni?"
# QUERY = "Milyen ciszplatin hatóanyagú gyógyszert tudok kiírni?"
# QUERY = "Mennyibe kerül a syntroxine?"
# QUERY = "Milyen gyógyszert lenne érdemes felírni láz esetén?"
# QUERY = "Milyen kiszerelésben kapható az egilok tabletta?"
# QUERY = "Ki forgalmazza a perdox oldatot?"
# QUERY = "Mivel lehet kiváltani a Matrifent?"
# QUERY = "Ki írhatja fel a letrozol pharcet?"
QUERY = "Milyen betegségek esetén alkalmazható a Brusimp?"
# QUERY = "Mi a calcium fluoratum ATC kódjának értelmezése?"

In [5]:
QUERY = "Milyen Algoflex típusokat ismersz?"

In [6]:
sql_gen_messages = [
                {"role": "system", "content" : sql_system_message.format(system_prompt=SQL_SYSTEM_PROMPT, 
                                                                    table_info=SCHEMA_PROMPT)},
                {"role": "user", "content": question_message.format(question=QUERY)}
                ]

In [7]:
full_response = client.chat.completions.create(
    model="TextToSql_Pupha",
    messages=sql_gen_messages,
    temperature=0.0,
    max_tokens=1024,
    logprobs=True
)
pprint(full_response.choices[0].message.content)

("SELECT OEP_NEV FROM GYOGYSZ WHERE OEP_NEV ILIKE '%Algoflex%' ORDER BY "
 'OEP_NEV LIMIT 10')


In [23]:
pprint(full_response)

ChatCompletion(id='chatcmpl-9HDdIkwCu53nDDfgiXArbRYdLhK6q', choices=[Choice(finish_reason='stop', index=0, logprobs=ChoiceLogprobs(content=[ChatCompletionTokenLogprob(token='SELECT', bytes=[83, 69, 76, 69, 67, 84], logprob=-0.010786171, top_logprobs=[]), ChatCompletionTokenLogprob(token=' EU', bytes=[32, 69, 85], logprob=-1.3006307, top_logprobs=[]), ChatCompletionTokenLogprob(token='IND', bytes=[73, 78, 68], logprob=-0.0014440456, top_logprobs=[]), ChatCompletionTokenLogprob(token='IK', bytes=[73, 75], logprob=-2.6968896e-06, top_logprobs=[]), ChatCompletionTokenLogprob(token='AC', bytes=[65, 67], logprob=0.0, top_logprobs=[]), ChatCompletionTokenLogprob(token='IO', bytes=[73, 79], logprob=-4.3202e-07, top_logprobs=[]), ChatCompletionTokenLogprob(token='K', bytes=[75], logprob=-2.9352968e-06, top_logprobs=[]), ChatCompletionTokenLogprob(token='.LE', bytes=[46, 76, 69], logprob=-0.0020958956, top_logprobs=[]), ChatCompletionTokenLogprob(token='IR', bytes=[73, 82], logprob=-1.9361265e-0

In [24]:
logprobs = [x.logprob for x in full_response.choices[0].logprobs.content]
calculate_confidence_score(logprobs)

98.05

In [8]:
db_connection = connect_to_db(str(os.environ['db_host']), str('pupha'), str(os.environ['db_user']), str(os.environ['db_password']))
query_answer = read_db(full_response.choices[0].message.content,db_connection)
close_connection(db_connection)

Connection established


In [9]:
query_answer

,0
0,ALGOFLEX BABY 20 MG/ML BELSŐLEGES SZUSZPENZIÓ
1,ALGOFLEX DUO 400 MG/100 MG FILMTABLETTA
2,ALGOFLEX DUO 400 MG/100 MG FILMTABLETTA
3,ALGOFLEX DUO 400 MG/100 MG FILMTABLETTA
4,ALGOFLEX DUO 400 MG/100 MG FILMTABLETTA
5,ALGOFLEX FORTE DOLO 400 MG FILMTABLETTA
6,ALGOFLEX FORTE DOLO 400 MG FILMTABLETTA
7,ALGOFLEX FORTE DOLO 400 MG FILMTABLETTA
8,ALGOFLEX FORTE DOLO 400 MG FILMTABLETTA
9,ALGOFLEX FORTE DOLO 400 MG FILMTABLETTA


In [35]:
db_connection = connect_to_db(str(os.environ['db_host']), str("pupha"), str(os.environ['db_user']), str(os.environ['db_password']))
print(read_db("SELECT BNOKODOK.LEIRAS FROM GYOGYSZ JOIN EUHOZZAR ON GYOGYSZ.ID = EUHOZZAR.GYOGYSZ_ID JOIN EUPONTOK ON EUHOZZAR.EUPONT_ID = EUPONTOK.ID JOIN BNOHOZZAR ON BNOHOZZAR.EUPONT_ID = EUPONTOK.ID JOIN BNOKODOK ON BNOHOZZAR.BNO_ID = BNOKODOK.ID WHERE GYOGYSZ.OEP_NEV LIKE '*BRUSIMP*'",db_connection))
close_connection(db_connection)

Connection established
Empty DataFrame
Columns: []
Index: []


In [14]:
query_answer.drop_duplicates()

,0,1
0,1012,Sugártherápia
1,10172,Sugártherápia
2,10526,Klinikai onkológia
3,10547,Klinikai onkológia
4,10547,Sugártherápia
5,10606,Sugártherápia
6,10614,Klinikai onkológia
7,10614,Sugártherápia
8,10922,Klinikai onkológia
9,11567,Sugártherápia


In [25]:
messages = [
                {"role": "system", "content" : text_system_message.format(system_prompt=DB_TO_TEXT_SYSTEM_PROMPT, 
                                                                    sql_data=query_answer.drop_duplicates())},
                {"role": "user", "content": question_message.format(question=QUERY)}
            ]

In [26]:
response = client.chat.completions.create(
    model="TextToSql_Pupha",
    messages=messages,
    temperature=0.5,
    max_tokens=1024,
    logprobs=True
)
pprint(response.choices[0].message.content)

'Nem tudom.'
